In [23]:
#import the packages
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import ta
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [24]:
today = date.today()
ticker1 = 'ES=F' #'^VIX'
start_date = '2019-01-02'
end = today
df = pdr.get_data_yahoo(ticker1, start=start_date, end=today)
df['RSI'] = ta.momentum.rsi(df.Close, window=14)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,RSI
Date,,,,,,,
2019-01-02,2508.00,2521.25,2452.25,2511.00,2511.00,1969697,NaN
2019-01-03,2482.25,2493.50,2443.25,2447.75,2447.75,2257341,NaN
2019-01-04,2451.75,2539.25,2438.50,2531.25,2531.25,2350836,NaN
2019-01-07,2537.25,2567.50,2523.25,2550.50,2550.50,1632131,NaN
2019-01-08,2553.25,2581.25,2547.50,2572.50,2572.50,1664892,NaN
...,...,...,...,...,...,...,...
2024-07-15,5681.00,5718.75,5665.50,5683.00,5683.00,1701349,75.239349
2024-07-16,5688.75,5721.25,5673.00,5717.25,5717.25,1485113,77.742451
2024-07-17,5715.00,5717.75,5631.75,5639.00,5639.00,2059862,62.257309


In [25]:
df_rsi = df[['Close', 'RSI']].copy()
df_rsi = df_rsi.dropna()
df_rsi = df_rsi.round(2)
df_rsi

,Close,RSI
Date,,
2019-01-22,2632.00,66.20
2019-01-23,2638.25,67.19
2019-01-24,2634.00,65.78
2019-01-25,2663.50,70.41
2019-01-28,2642.00,63.64
...,...,...
2024-07-15,5683.00,75.24
2024-07-16,5717.25,77.74
2024-07-17,5639.00,62.26


In [31]:
contracts = 1
multiplier = 5
last_signal = float(df_rsi.Close.iloc[0])
delta = last_signal * 0.1
date = df_rsi.index[0].strftime('%Y-%m-%d')
print(f"d: {date}, Buy {contracts} contracts at {last_signal}")
p_and_l = []
for x in df_rsi.index:
    date = x.strftime('%Y-%m-%d')
    close = float(df_rsi.loc[x, 'Close'])
    #print(date, close)
    if close < ( last_signal - delta ):
        p_and_l.append(contracts * (close - last_signal) * multiplier)   
        contracts += 1
        print(f"d: {date}, Buy {contracts} contracts at {close}")
        last_signal = close
    elif close > ( last_signal + delta ):
        p_and_l.append(contracts * (close - last_signal) * multiplier)   
        contracts -= 1
        if contracts == 0:
            contracts = 1
        if contracts == 1:
            delta = last_signal * 0.1
        print(f"d: {date}, Sell {contracts} contracts at {close}")
        last_signal = close

print(f'p&L: {p_and_l}')
print(f"Profit and Loss: {sum(p_and_l)}")
    

d: 2019-01-22, Buy 1 contracts at 2632.0
d: 2019-04-05, Sell 1 contracts at 2896.0
d: 2019-12-12, Sell 1 contracts at 3168.0
d: 2020-03-09, Buy 2 contracts at 2747.75
d: 2020-03-16, Buy 3 contracts at 2416.25
d: 2020-04-08, Sell 2 contracts at 2735.0
d: 2020-05-27, Sell 1 contracts at 3035.5
d: 2020-08-05, Sell 1 contracts at 3316.0
d: 2020-11-16, Sell 1 contracts at 3623.0
d: 2021-03-15, Sell 1 contracts at 3967.5
d: 2021-07-02, Sell 1 contracts at 4342.75
d: 2021-12-27, Sell 1 contracts at 4782.25
d: 2022-01-26, Buy 2 contracts at 4341.5
d: 2022-05-19, Buy 3 contracts at 3897.75
d: 2023-06-12, Sell 2 contracts at 4342.75
d: 2023-12-18, Sell 1 contracts at 4793.0
d: 2024-03-19, Sell 1 contracts at 5241.75
p&L: [1320.0, 1360.0, -2101.25, -3315.0, 4781.25, 3005.0, 1402.5, 1535.0, 1722.5, 1876.25, 2197.5, -2203.75, -4437.5, 6675.0, 4502.5, 2243.75]
Profit and Loss: 20563.75
